In [4]:
# !pip install qiskit-nature
!poetry run pip install networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.3 MB/s eta 0:00:00 0:00:01


In [18]:
from math import pi
import json
from pathlib import Path
from math import sqrt
from typing import *

import numpy as np
from dotenv import load_dotenv
import rustworkx as rx
import networkx as nx
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.second_q.problems import LatticeModelProblem
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Sampler
from qiskit.quantum_info import Pauli
from qiskit.result import QuasiDistribution
from qiskit_algorithms import QAOA as QAOASolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from process_bigraph import Process, Step
from process_bigraph import pp

from bsp import registrar


load_dotenv("../config.env")

True

In [6]:
from process_bigraph import Process, Step


class RQAOA(Process):
    pass


class VariableReduction(Process):
    """Run reduce_variables() workflow here"""
    pass


class VariableChecker(Step):
    """takes N as input state and checks if N == Ncrit"""
    pass


from qiskit.quantum_info import SparsePauliOp
def build_max_cut_paulis(graph: rx.PyGraph) -> list[tuple[str, float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        paulis = ["I"] * len(graph)
        paulis[edge[0]], paulis[edge[1]] = "Z", "Z"

        weight = graph.get_edge_data(edge[0], edge[1])

        pauli_list.append(("".join(paulis)[::-1], weight))

    return pauli_list


class MolecularQAOA(Process):
    config_schema = {
        'nuclei': 'list[list[float]]'
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        # position vectors of all nuclei involved in solving
        self.R = self.config['nuclei']

In [7]:
# from bsp import registrar
# CORE = registrar.core
# 
# rqaoa = RQAOA(
#     config={
#         'n_nodes': 5,
#         'edge_list': [(0, 1, 1.0), (0, 2, 1.0), (0, 4, 1.0), (1, 2, 1.0), (2, 3, 1.0), (3, 4, 1.0)] 
#     },
#     core=CORE
# )

"""R is an array of arrays of shape: (n, 3)
    where n is the number of nuclei involved in the hamiltonian, and 
    3 represent position vector dimesions (x, y, z) for each n
"""

'R is an array of arrays of shape: (n, 3)\n    where n is the number of nuclei involved in the hamiltonian, and \n    3 represent position vector dimesions (x, y, z) for each n\n'

In [37]:
class GroverProcess(Process):
    config_schema = {
        "target": "string"
    }

    def __init__(self, config, core):
        super().__init__(config, core)





In [38]:
CORE = registrar.core

registrar.register_process("qaoa", QAOA, verbose=True)

Successfully registered <class '__main__.QAOA'> to address: qaoa


In [12]:
from bsp.io import read_xyz

atom_xyz = read_xyz('../../assets/experiments/ferroxidase_center.xyz')

In [13]:
atom_xyz

'Fe 0.0 0.0 0.0; Fe 2.5 0.0 0.0; O 1.25 0.5 0.0; O 1.25 -0.5 0.0; N 0.5 1.5 0.0; C -0.5 -1.5 0.0; C 3.0 1.0 0.0; O 3.5 1.5 0.0'

In [46]:
import os
from bsp.data_model import BaseClass
import numpy as np
from dataclasses import dataclass


class DiracNotation(np.ndarray):
    def __new__(cls, values: list[complex]):
        return np.asarray(values).view(cls)


class Bra(DiracNotation):
    pass


class Ket(DiracNotation):
    def bra(self) -> Bra:
        ket_value = self.view()
        return Bra(np.conj(ket_value).T)


class XYZElements(list):
    pass


class XYZAtom(str):
    def __new__(cls, *args, **kwargs):
        return str.__new__(cls, *args, **kwargs)

    @property
    def elements(self, unique: bool = False) -> XYZElements:
        items = self.split()
        elements = XYZElements([line[0] for line in [ln.strip() for ln in self.split(";")]])
        return list(set(elements)) if unique else elements


@dataclass
class Atom(BaseClass):
    name: str
    x: float
    y: float
    z: float


@dataclass
class Molecule(BaseClass):
    xyz_filepath: Optional[os.PathLike[str]] = None
    atoms: Optional[List[Atom]] = None

    def __post_init__(self):
        if self.atoms is None and self.xyz_filepath is not None:
            self.atoms = self.read_xyz(self.xyz_filepath)

    @property
    def xyz(self) -> str:
        xyz_mol = []
        for atom in self.atoms:
            xyz_atom = f"{atom.name} {atom.x} {atom.y} {atom.z}"
            xyz_mol.append(xyz_atom)
        return "; ".join(xyz_mol)

    def read_xyz(self, xyz_filepath: Optional[os.PathLike[str]] = None) -> List[Atom]:
        atoms = []
        fp = xyz_filepath or self.xyz_filepath
        with open(fp) as f:
            file_contents = f.read().splitlines()
            for i, line in enumerate(file_contents):
                atom = Atom(*line.split())
                atoms.append(atom)
        return atoms







In [51]:
# atom_xyz = 'Fe 0.0 0.0 0.0; Fe 2.5 0.0 0.0; O 1.25 0.5 0.0; O 1.25 -0.5 0.0; N 0.5 1.5 0.0; C -0.5 -1.5 0.0; C 3.0 1.0 0.0; O 3.5 1.5 0.0'
mol = Molecule(
    xyz_filepath='/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/assets/experiments/ferroxidase_center.xyz'
    # atoms=[
    #     Atom(name='Fe', x=0.0, y=0.0, z=0.0),
    #     Atom(name='Fe', x=2.5, y=0.0, z=0.0),
    #     Atom(name='O', x=1.25, y=-0.5, z=0.0)
    # ]
)

In [52]:
mol.xyz

'Fe 0.0 0.0 0.0; Fe 2.5 0.0 0.0; O 1.25 0.5 0.0; O 1.25 -0.5 0.0; N 0.5 1.5 0.0; C -0.5 -1.5 0.0; C 3.0 1.0 0.0; O 3.5 1.5 0.0'

In [53]:
mol.atoms

[Atom(name='Fe', x='0.0', y='0.0', z='0.0'),
 Atom(name='Fe', x='2.5', y='0.0', z='0.0'),
 Atom(name='O', x='1.25', y='0.5', z='0.0'),
 Atom(name='O', x='1.25', y='-0.5', z='0.0'),
 Atom(name='N', x='0.5', y='1.5', z='0.0'),
 Atom(name='C', x='-0.5', y='-1.5', z='0.0'),
 Atom(name='C', x='3.0', y='1.0', z='0.0'),
 Atom(name='O', x='3.5', y='1.5', z='0.0')]

In [54]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

In [55]:
driver = PySCFDriver(
    atom=mol.xyz,
    basis="LANL2DZ", # "sto3g",
    charge=0,
    spin=10,
    unit=DistanceUnit.ANGSTROM,
)

In [56]:
es_problem = driver.run()

QiskitNatureError: 'Failed to build the PySCF Molecule object.'